### Functions

In [1]:
import astra

def get_sinogram_slice(vol_slice, angles):
    """
    Get a sinogram from a 3D dataset slice.
    
    Parameters
    ----------
    slice : 2D numpy.ndarray
        Slice
        
    Returns
    -------
    numpy.ndarray
        2D sinogram.
        
    """
    vol_geom = astra.create_vol_geom(vol_slice.shape[1], vol_slice.shape[0])
    proj_geom = astra.create_proj_geom('parallel', 1.0, vol_slice.shape[1], angles)

    projector_id = astra.create_projector('cuda', proj_geom, vol_geom)
    _, sino  = astra.create_sino(vol_slice, projector_id)

    return sino

### Loading data 

In [2]:
import h5py
import numpy as np
import napari

def find_datasets_with_dim_3(file, group=None, path="", results=None):
    """
    Find all datasets with 3 dimensions in a HDF5 file.       
    """
    if results is None:
        results = []

    if group is None:
        group = file

    for key in group:
        item = group[key]
        current_path = f"{path}/{key}"
        if isinstance(item, h5py.Group):
            find_datasets_with_dim_3(
                file, group=item, path=current_path, results=results
            )
        elif isinstance(item, h5py.Dataset):
            if len(item.shape) == 3:
                results.append(
                    (current_path, item.shape)
                ) 
    return results

projs_path = r"D:\SOLEIL0125\025_mdb13_nocell_nocell_2d\025_mdb13_nocell_nocell_2d\025_mdb13_nocell_nocell_2d.nxs"
dark_path = r"D:\SOLEIL0125\025_mdb13_nocell_nocell_2d\025_mdb13_nocell_nocell_2d\post_dark.nxs"
flat_paths = r"D:\SOLEIL0125\025_mdb13_nocell_nocell_2d\025_mdb13_nocell_nocell_2d\post_ref.nxs"

with h5py.File(projs_path, 'r') as file:
    projs_key = find_datasets_with_dim_3(file)[0][0]
    projs = np.array(file[projs_key])[:, 150:800]
print("Projections shape:", projs.shape)

with h5py.File(dark_path, 'r') as file:
    dark_key = find_datasets_with_dim_3(file)[0][0]
    dark = np.mean(np.array(file[dark_key]), axis=0)[150:800]
print("Dark shape:", dark.shape)

flat = []
with h5py.File(flat_paths, 'r') as file:
    flat_key = find_datasets_with_dim_3(file)[0][0]
    flat = np.mean(np.array(file[flat_key]), axis=0)[150:800]
print("Flat shape:", flat.shape)

Projections shape: (2850, 650, 1472)
Dark shape: (650, 1472)
Flat shape: (650, 1472)


In [3]:
### Get Center of Rotation from csv file


import pandas as pd
import os

csv_path = r"C:\Users\clement.capdeville\Documents\GitHub\SOLEIL25\part_info.csv"
part_info = pd.read_csv(csv_path)
filtered_info = part_info.loc[part_info["Path"] == os.path.basename(projs_path).split(".")[0]]
CoR = filtered_info["CoR"].values[0] if not filtered_info.empty else None
angle_padding = filtered_info["Angle"].values[0] if not filtered_info.empty else None

print("CoR:", CoR)
print("Angle padding:", angle_padding)

CoR: 287.5329942270546
Angle padding: 0.1264044943820224


### Preprocessing

In [4]:
projs = (projs - dark) / (flat - dark)

### Raw Projections

In [5]:
import numpy as np
from tqdm import tqdm

from src.reco_plugin.processing.reconstruction import create_angles, reconstruct_from_sinogram_slice, create_disk_mask
from src.reco_plugin.processing.sinogram import create_sinogram

sinogram = create_sinogram(projs, int(2*CoR))
angles = create_angles(sinogram, end=np.pi)
raw_reconstruction = np.zeros((sinogram.shape[0], sinogram.shape[2], sinogram.shape[2]), dtype=np.float32)
mask = create_disk_mask(sinogram)

for i in tqdm(range(sinogram.shape[0]), desc="Reconstructing"):
    raw_reconstruction[i] = reconstruct_from_sinogram_slice(sinogram[i], angles) * mask

Reconstructing: 100%|██████████| 650/650 [01:47<00:00,  6.04it/s]


In [ ]:
import napari

viewer = napari.Viewer()
viewer.add_image(raw_reconstruction, name="Reconstruction", colormap="gray")

In [32]:
import tifffile as tiff

tiff.imwrite(r"raw_reconstruction.tif", raw_reconstruction, imagej=True)

c:\Users\clement.capdeville\.conda\envs\napari-env\Lib\site-packages\tifffile\tifffile.py:3784: UserWarning: <tifffile.TiffWriter 'raw_reconstruction.tif'> truncating ImageJ file
  warnings.warn(


### 1rst Paganin : Émail

In [6]:
from src.reco_plugin.processing.phase import paganin_filter

energy = 40 # keV
pixel = 12e-6 # m
effective_pixel = pixel # m
distance = 1.2 # m
db = 1500

pagain_proj = paganin_filter(projs, energy, pixel, effective_pixel, distance, db)['paganin']

Processing Paganin: 100%|██████████| 2850/2850 [00:49<00:00, 57.46it/s]


In [ ]:
import napari

viewer = napari.Viewer()
viewer.add_image(pagain_proj, name="Pagani Projection", colormap="gray")

In [7]:
import numpy as np
import cupy as cp
from tqdm import tqdm

from src.reco_plugin.processing.reconstruction import create_angles, reconstruct_from_sinogram_slice, create_disk_mask
from src.reco_plugin.processing.sinogram import create_sinogram
from src.reco_plugin.processing.phase import unsharp_mask

coeff = 0.5
sigma = 2

sinogram = create_sinogram(pagain_proj, int(2*CoR))
angles = create_angles(sinogram, end=np.pi)
pag_reconstruction = np.zeros((sinogram.shape[0], sinogram.shape[2], sinogram.shape[2]), dtype=np.float32)
mask = create_disk_mask(sinogram)

for i in tqdm(range(sinogram.shape[0]), desc="Reconstructing"):
    slice_ = reconstruct_from_sinogram_slice(sinogram[i], angles) * mask
    pag_reconstruction[i] = slice_

Reconstructing: 100%|██████████| 650/650 [01:50<00:00,  5.86it/s]


In [ ]:
import napari

viewer = napari.Viewer()
viewer.add_image(pag_reconstruction, name="Paganin 1 Projection", colormap="gray")

In [8]:
import cupy as cp
from tqdm import tqdm

from cupyx.scipy.ndimage import binary_dilation, binary_erosion

email = np.zeros_like(pag_reconstruction)

for i in tqdm(range(pag_reconstruction.shape[0]), desc="Reconstructing"):
    slice_ = cp.asarray(pag_reconstruction[i])
    slice_ = cp.where(slice_ > 25, 1, 0)
    slice_ = binary_dilation(slice_)
    slice_ = binary_dilation(slice_)
    email[i] = slice_.get()

viewer = napari.Viewer()
viewer.add_image(email, name="Email", colormap="gray")
viewer.add_image(pag_reconstruction, name="Paganin 1 Projection", colormap="gray")

Reconstructing: 100%|██████████| 650/650 [00:09<00:00, 67.01it/s]
napari.manifest -> 'multipaganinproject' could not be imported: The name field in the manifest ('multipaganin-plugin') must match the package name ('multipaganinproject')


<Image layer 'Paganin 1 Projection' at 0x26b43239160>

In [9]:
final = np.zeros_like(raw_reconstruction)

final[email != 0] = pag_reconstruction[email != 0]

### Step 2

In [ ]:
raw2_reconstruction = np.copy(raw_reconstruction)
raw2_reconstruction[email != 0] = 0.00015

viewer = napari.Viewer()
viewer.add_image(raw2_reconstruction, name="Reconstruction", colormap="gray")

<Image layer 'Reconstruction' at 0x26ef0ba3500>

In [11]:
pag2_sino = np.zeros_like(sinogram)

for i in tqdm(range(raw2_reconstruction.shape[0]), desc="Reconstructing"):
    sino_ = get_sinogram_slice(raw2_reconstruction[i], angles)
    pag2_sino[i] = sino_

projs = np.swapaxes(pag2_sino, 0, 1)

Reconstructing: 100%|██████████| 650/650 [00:37<00:00, 17.31it/s]


In [ ]:
viewer = napari.Viewer()
viewer.add_image(projs, name="Pagani 2 Projection", colormap="gray")

In [12]:
from src.reco_plugin.processing.phase import paganin_filter

energy = 40 # keV
pixel = 12e-6 # m
effective_pixel = pixel # m
distance = 1.2 # m
db = 1500

pagain_proj = paganin_filter(projs, energy, pixel, effective_pixel, distance, db)['paganin']

Processing Paganin: 100%|██████████| 1425/1425 [00:51<00:00, 27.80it/s]


In [13]:
import numpy as np
import cupy as cp
from tqdm import tqdm

from src.reco_plugin.processing.reconstruction import create_angles, reconstruct_from_sinogram_slice, create_disk_mask
from src.reco_plugin.processing.sinogram import create_sinogram
from src.reco_plugin.processing.phase import unsharp_mask

coeff = 0.5
sigma = 2

sinogram = np.swapaxes(pagain_proj, 0, 1)
angles = create_angles(sinogram, end=np.pi)
mask = create_disk_mask(sinogram)

for i in tqdm(range(sinogram.shape[0]), desc="Reconstructing"):
    slice_ = reconstruct_from_sinogram_slice(sinogram[i], angles) * mask
    pag_reconstruction[i] = slice_

Reconstructing: 100%|██████████| 650/650 [01:43<00:00,  6.31it/s]


In [ ]:
import napari

viewer = napari.Viewer()
viewer.add_image(pag_reconstruction, name="Paganin 2 Projection", colormap="gray")

In [14]:
import cupy as cp
from tqdm import tqdm

from cupyx.scipy.ndimage import binary_dilation, binary_erosion

dentine_os = np.zeros_like(pag_reconstruction)

for i in tqdm(range(pag_reconstruction.shape[0]), desc="Reconstructing"):
    slice_ = cp.asarray(pag_reconstruction[i])
    slice_ = cp.where(slice_ > 5, 1, 0)
    slice_ = binary_dilation(slice_)
    slice_ = binary_dilation(slice_)
    dentine_os[i] = slice_.get()

viewer = napari.Viewer()
viewer.add_image(dentine_os, name="Dentine/Os", colormap="gray")
viewer.add_image(pag_reconstruction, name="Paganin 1 Projection", colormap="gray")

Reconstructing: 100%|██████████| 650/650 [00:08<00:00, 77.54it/s]


<Image layer 'Paganin 1 Projection' at 0x26ef8280650>

In [15]:
final[dentine_os != 0] = pag_reconstruction[dentine_os != 0]

In [16]:
viewer = napari.Viewer()
viewer.add_image(final, name="Reconstruction", colormap="gray")

<Image layer 'Reconstruction' at 0x272726ffbc0>

### Step 3

In [26]:
raw2_reconstruction[dentine_os != 0] = 0.0001

viewer = napari.Viewer()
viewer.add_image(raw2_reconstruction, name="Reconstruction", colormap="gray")

<Image layer 'Reconstruction' at 0x276e566f1d0>

In [27]:
pag2_sino = np.zeros_like(sinogram)

for i in tqdm(range(raw2_reconstruction.shape[0]), desc="Reconstructing"):
    sino_ = get_sinogram_slice(raw2_reconstruction[i], angles)
    pag2_sino[i] = sino_

projs = np.swapaxes(pag2_sino, 0, 1)

Reconstructing: 100%|██████████| 650/650 [00:41<00:00, 15.59it/s]


In [19]:
viewer = napari.Viewer()
viewer.add_image(projs, name="Pagani 2 Projection", colormap="gray")

<Image layer 'Pagani 2 Projection' at 0x272ebccbc20>

In [28]:
from src.reco_plugin.processing.phase import paganin_filter

energy = 40 # keV
pixel = 12e-6 # m
effective_pixel = pixel # m
distance = 1.2 # m
db = 1500

pagain2_proj = paganin_filter(projs, energy, pixel, effective_pixel, distance, db)['paganin']

Processing Paganin: 100%|██████████| 1425/1425 [00:39<00:00, 36.26it/s]


In [29]:
import numpy as np
import cupy as cp
from tqdm import tqdm

from src.reco_plugin.processing.reconstruction import create_angles, reconstruct_from_sinogram_slice, create_disk_mask
from src.reco_plugin.processing.sinogram import create_sinogram
from src.reco_plugin.processing.phase import unsharp_mask

coeff = 0.5
sigma = 2

sinogram = np.swapaxes(pagain2_proj, 0, 1)
angles = create_angles(sinogram, end=np.pi)
mask = create_disk_mask(sinogram)

for i in tqdm(range(sinogram.shape[0]), desc="Reconstructing"):
    slice_ = reconstruct_from_sinogram_slice(sinogram[i], angles) * mask
    pag_reconstruction[i] = slice_

Reconstructing: 100%|██████████| 650/650 [01:46<00:00,  6.07it/s]


In [30]:
import napari

viewer = napari.Viewer()
viewer.add_image(pag_reconstruction, name="Paganin 2 Projection", colormap="gray")

<Image layer 'Paganin 2 Projection' at 0x276fab771a0>

In [23]:
# pag_reconstruction[final != 0] = final[final != 0]
test = np.copy(pag_reconstruction)
test[final != 0] = final[final != 0]

In [24]:
viewer = napari.Viewer()
viewer.add_image(test, name="Reconstruction", colormap="gray")

<Image layer 'Reconstruction' at 0x276bd0fd4c0>